In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import datetime
from dateutil.relativedelta import relativedelta

%matplotlib inline

note : 

description plusieurs rubrique -> bonne idée l'extraction des données.
année et age qui influence le prix et la date de mise en ligne. calcul de l'age pour explication du prix .
-> eda à faire 


Prochaine question : comment faire une bonne vérification map pour algo ,le word2vec  LES VERSIONS. 

#Exploration de la donnée : 
pour arriver à avoir un bon algo. Il faut une exploration de donnée pour savoir quelle variable va plus influence que d'autre le prix du véhicule. 

In [0]:
df = pd.read_csv("Data_cars.csv")

ParserError: ignored

In [0]:
df.head()

In [0]:
df.shape

In [0]:
df.info()

In [0]:
df['Description'].head()

In [0]:
df.isnull().sum()

On ne voit pas de valeur non null donc il y a pas de nettoyage particulier à faire à ce niveau là. 

On essaie de trouver des variables intéressante.

In [0]:
df['Online'] = pd.to_datetime(df['Online'], format='%d/%m/%Y à %Hh%M', errors='ignore')

#visualisation de la donnée


# Nouvelle section

In [0]:
pie = ['Fuel']

fig = plt.figure(figsize = (15,15))
axes = 220
for p in pie:
    axes += 1
    fig.add_subplot(axes)
    plt.pie(df[p].value_counts(), labels=df[p].unique(),autopct='%1.1f%%', shadow=True, startangle=45);
    plt.title(p.upper())
plt.show()

In [0]:
plt.figure(figsize=(20,15))
ax = sns.countplot(x='Model_year',data=df);
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right",fontsize=6);


In [0]:
plt.figure(figsize=(20,15))
ax = sns.countplot(x='Make',data=df);
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right",fontsize=6);


In [0]:
#create correlation
corr = df.corr(method = 'pearson')

#convert correlation to numpy array
mask = np.array(corr)

#to mask the repetitive value for each pair
mask[np.tril_indices_from(mask)] = False
fig, ax = plt.subplots(figsize = (15,12))
fig.set_size_inches(15,15)
sns.heatmap(corr, mask = mask, vmax = 0.9, square = True, annot = True)


on voit que le prix le model et année influence le modele.
Mais la description est remplie d'élément utilisable on va donc retirer ces éléments pour améliorer le modéle. 

Préparation à la création de l'algo:

In [0]:
df.Description.iloc[0]

In [0]:
# puissance_fiscale  
# portes 
# couleur
# description plus 
#cycle moteur 
#puissance du véhicule
#age seront les éléments que devront prendre en compte l'algo

In [0]:
df['Mileage'] = pd.to_numeric(df['Mileage']) 

In [0]:
d = {'puissance_fiscale':[],
    'portes':[], 
     'version':[],
     'couleur':[], 
     'options':[]
    }


for x in df['Description']:
    match = re.search(r"puissance_fiscale:(.*?),", x)
    # converts string to int 
    d['puissance_fiscale'].append(int(match.group(1)))
    
    
    match = re.search(r'version:(.*?),', x)
    d['version'].append(match.group(1))
    
    match = re.search(r"portes:(.*?),", x)
    #converts string to int
    
    try:
        d['portes'].append(float(match.group(1).strip()))
        
    except: 
        d['portes'].append(None)
        
    match = re.search(r"couleur:(?P<name>[A-Za-z\t .]+)", x)
    d['couleur'].append(match.group(1))

    match = re.search(r"options:(.*?),", x)
    d['options'].append(match.group(1))

In [0]:
d['couleur'] = [x.lower() for x in d['couleur']]

In [0]:
df1 = pd.DataFrame(d, columns=['puissance_fiscale', 'portes', 'version','couleur', 'options'])

In [0]:
df1.head()

In [0]:
df = pd.concat([df, df1], axis=1, sort=False)

In [0]:
df = df.drop('Description', axis=1)

In [0]:
#on commence par l'age 
d = {'age':[]}

for x in df['Online']:

    y = relativedelta(x(),Model_year).years
    d['age'].append(y)


In [0]:
df1 = pd.DataFrame(d, columns=['age'])

In [0]:
df = pd.concat([df, df1], axis=1, sort=False)

In [0]:
df.head()

In [0]:
df.corr()

In [0]:
texts = df['couleur'].to_numpy()

In [0]:
d = {
    'cylindre': [], 
     'cheveaux': []
    }

for x in df['version']:
    try: 
        
        match = re.search(r'[+-]?(\d+(\.\d*)|\.\d+)([eE][+-]?\d+)?', x)
        d['cylindre'].append(float(match.group(1)))
    
    except: 
        d['cylindre'].append(None)
        
    try: 
        
        match = re.findall(r'([0-9]{2,3})', x)
        temp = [int (n) for n in match]
        d['cheveaux'].append(max(temp))
        
    except: 
        
        d['cheveaux'].append(None)
        
        


In [0]:
import requests
from bs4 import BeautifulSoup


def getCylinder(make, model):

    URL = 'https://www.autoevolution.com/' + make.lower() +'/'+ str(model).lower() +'/'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    x = soup.find_all('span', class_='col-green2')[0].text
    print(x)
    match = re.search(r'[+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?', x)
    print(match.group(1))
    
    return match.group(1)

In [0]:
df.head()

In [0]:
# objectif : test Linear Regression,Gradient Boost,Random Forest,Light GBM, XGBoost, KMeans + LinReon

In [0]:
#on test random forest


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [0]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [0]:
df = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
df.fit(train_features, train_labels);

In [0]:
predictions = df.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
Mean Absolute Error: 3.83 degrees.

In [0]:
#Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
